In [1]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import HoverTool

# Libraries for the slider
from bokeh.models.annotations import Label
from bokeh.layouts import column
from bokeh.models import CustomJS, Slider, ColumnDataSource

# Libararies for the tabs
from bokeh.models.widgets import Panel, Tabs

# Safe graphes
from bokeh.io import export_svg

# Plotting the graphs in the Jupyter notebook
output_notebook()

Loading BokehJS ...

In [2]:
# General set up formatting the database for our needs

df = pd.read_csv('weatherAUS.csv')
cities = df.Location.unique()

# Get Day and Year from each entry
df['Day'] = df.Date.apply(lambda x : x.split("-")[2])
df['Month'] = df.Date.apply(lambda x : x.split("-")[1])
df['Year'] = df.Date.apply(lambda x : x.split("-")[0])

# Get the Coordinates for each Location
df_locations = pd.read_csv('locationsDegree_formatted.csv', index_col=0)

# Transform the coordinates using the EPSG 3857 System because it worked for Darwin. 
df_locations.Longitude = df_locations.Longitude * 20037508.34 / 180
df_locations.Latitude = np.log(np.tan((90 + df_locations.Latitude) * np.pi / 360)) / (np.pi / 180)
df_locations.Latitude = df_locations.Latitude.apply(lambda x : x * 20037508.34 / 180)

In [3]:
# Decide what kind of Data you wanna collect: 
METHOD= 'max' # Possible values 'min', 'max', 'mean', 'sum'
WEATHERCONDITION = 'Rainfall' # Possible values 'MaxTemp', 'MinTemp', 'Rainfall'
# 'sum' should only be used with 'Rainfall'

In [4]:
# Parameter specific set up
# Compute the maximal temperature for months
df_advanced = df.groupby(['Location', 'Year', 'Month'], as_index = False).agg({WEATHERCONDITION : METHOD})
                                               #'MinTemp' : ['mean', 'max', 'min'],
                                               #'Rainfall': ['mean', 'max', 'min']})

# Create DataFrame with important information. For the February Tab.
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
months_name = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']

df_months = {}
for month in months:
    df_months[month] = df_advanced[df_advanced.Month == month]
    df_months[month] = df_months[month].merge(right = df_locations, how='left', on='Location')

In [5]:
tabs = {}
for month in months:
    sources = {}
    labels = {}
    for city in cities:
        source = ColumnDataSource(df_months[month][df_months[month].Location == city])
        sources[city] = source
        label = Label(x = df_locations.loc[city, 'Longitude'], y = df_locations.loc[city, 'Latitude'], text = 'Test' , x_offset = 0.4, y_offset = 10)
        labels[city] = label

    slider = Slider(start=2009, end=2017, value=2009, step=1, title="Year Slider")

    callback = CustomJS(args=dict(sources=sources, labels=labels, slider=slider, cities=cities, wc=WEATHERCONDITION), code="""
        function makeEmojiTemp(x) {
                if (x < 30 ) { 
                    return '\U0001F642';
                } else if (30 <= x && x < 35) { 
                    return '\U0001F60E';
                } else if (35 <= x && x < 40) { 
                    return '\U0001F975';
                } else if (x > 40) { 
                    return '\U0001FAE0';
                } else {
                    return '';
                }
            }
        
        function makeEmojiRain(x) {
            if (x < 30 ) { 
                return '\U0001F324';
            } else if (30 <= x && x < 35) { 
                return '\U0001F325';
            } else if (35 <= x && x < 40) { 
                return '\U0001F326';
            } else if (x > 40) { 
                return '\U0001F327';
            } else {
                return '';
            }
        }
        
        
        for (let i = 0; i < cities.length; i++) {
            var data_from_source = sources[cities[i]].data;
            var mop = data_from_source[wc][slider.value - 2009];
            if (wc == 'Rainfall') {
                labels[cities[i]].text = makeEmojiRain(mop);
            } else {
                labels[cities[i]].text = makeEmojiTemp(mop);
            }
        }
    """)
    slider.js_on_change('value', callback)

    tooltips = [("Location", "@Location"), 
                ("(x, y)", "(@Longitude, @Latitude)")]

    # Initialization of a figure
    p = figure(x_range=(12500000, 17500000),
            y_range=(-6000000, -1400000), # y-axis range
            x_axis_type = 'mercator',      # type of the axis, in this case a mercator projection which is used to find one's way on an 
                                            # flat map of the Earth
            y_axis_type = 'mercator')      # // 

    tile = get_provider(Vendors.CARTODBPOSITRON)
    
    # Adding the tile to the figure
    p.add_tile(tile)     

    circle = p.circle(x = 'Longitude',           # x-axis
            y = 'Latitude',           # y-axis
            fill_color = 'blue', # circle fill color
            size = 10,           # circle size
            source = df_locations) # We use locations here so that we get each point only once!

    hover = HoverTool(tooltips = tooltips, renderers = [circle])

    # Displaying the figure
    p.add_tools(hover)
    for city in cities:
        p.add_layout(labels[city])

    layout = column(p, slider)
    tab = Panel(child = layout, title = months_name[int(month) - 1])
    tabs[month] = tab

t = Tabs(tabs = [tabs[x] for x in months])

show(t)

['RainFallSum.svg']